<a href="https://colab.research.google.com/github/yingchenjin/IANAL/blob/main/paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
from transformers import pipeline
import io
import re

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
uploaded = files.upload()

Saving GNULibGPL2.txt to GNULibGPL2.txt
Saving GNULGPL3.txt to GNULGPL3.txt
Saving GNULGPL2.txt to GNULGPL2.txt
Saving GNUGPL2.txt to GNUGPL2.txt
Saving GNUGPL3.txt to GNUGPL3.txt
Saving GNUAffero.txt to GNUAffero.txt
Saving Apache1.1.txt to Apache1.1.txt
Saving Apache2.0.txt to Apache2.0.txt
Saving Nethack.txt to Nethack.txt
Saving MIT.txt to MIT.txt


In [ ]:
all_paragraphs = []
for filename in uploaded.keys():
    content = uploaded[filename].decode('utf-8')
    # Split text into paragraphs
    paragraphs = [p.strip() for p in re.split(r'\n\s*\n', content) if p.strip()]
    all_paragraphs.extend(paragraphs)

print(f"Found {len(all_paragraphs)} paragraphs")

Found 342 paragraphs


In [ ]:
# vamsi = pipeline(
#     "text2text-generation",
#     model="Vamsi/T5_Paraphrase_Paws",
#     max_length=520
# )
humarin = pipeline(
    "text2text-generation",
    model="humarin/chatgpt_paraphraser_on_T5_base",
    max_length=520
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def paraphrase_paragraph(paragraph):
    try:
        # vamsio = vamsi(
        #     f"paraphrase: {paragraph}",
        #     num_return_sequences=1,
        #     num_beams=5,
        #     max_length=520,
        #     no_repeat_ngram_size=2
        # )[0]['generated_text']
        humarin6 = humarin(
            f"paraphrase: {paragraph}",
            num_return_sequences=1,
            do_sample = True,
            temperature = 0.6,
            max_length=520
        )[0]['generated_text']
        humarin7 = humarin(
            f"paraphrase: {paragraph}",
            num_return_sequences=1,
            do_sample = True,
            temperature = 0.7,
            max_length=520
        )[0]['generated_text']
        humarin8 = humarin(
            f"paraphrase: {paragraph}",
            num_return_sequences=1,
            do_sample = True,
            temperature = 0.8,
            max_length=520
        )[0]['generated_text']

        return {
            # 'vamsi': vamsio,
            'humarin6': humarin6,
            'humarin7': humarin7,
            'humarin8': humarin8
        }
    except Exception as e:
        print(f"Error paraphrasing: {str(e)}")
        return {'humarin6': '', 'humarin7': '', 'humarin8': ''}

In [ ]:
output_data = []

for para in all_paragraphs:

    # Store original paragraph's sentences
    original_sentences = sent_tokenize(para)
    paraphrases = paraphrase_paragraph(para)

    # Split paraphrased versions into sentences
    # vamsi_sentences = sent_tokenize(paraphrases['vamsi']) if paraphrases['vamsi'] else []
    humarin6_sentences = sent_tokenize(paraphrases['humarin6']) if paraphrases['humarin6'] else []
    humarin7_sentences = sent_tokenize(paraphrases['humarin7']) if paraphrases['humarin7'] else []
    humarin8_sentences = sent_tokenize(paraphrases['humarin8']) if paraphrases['humarin8'] else []

    # Create output row
    row = {
        'Original_Paragraph': para,
        'Original_Sentences': ' | '.join(original_sentences),
        # 'Vamsi_Paraphrase': ' | '.join(vamsi_sentences),
        'Humarin6': ' | '.join(humarin6_sentences),
        'Humarin7': ' | '.join(humarin7_sentences),
        'Humarin8': ' | '.join(humarin8_sentences)
    }

    output_data.append(row)

Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Create CSV with results
df = pd.DataFrame(output_data)
df = df[['Original_Paragraph', 'Original_Sentences', 'Humarin6', 'Humarin7', 'Humarin8']]
df.to_csv('paraphrases.csv', index=False)
files.download('paraphrases.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

legal_texts = uploaded


# Optionally, split the legal text into chunks (e.g., by paragraphs or sections)
# For this example, we'll split by two newlines which often separate paragraphs.
legal_chunks = []
for name, legal_text in legal_texts.items():  # directly iterate over the texts
    legal_text = legal_text.decode('utf-8')
    legal_chunks += [chunk.strip() for chunk in legal_text.split("\n") if len(chunk.strip()) > 100]
print("Number of legal chunks:", len(legal_chunks))

# --- Step 2: Load Forum Q&A Data ---
# Assuming your CSV has a column named "text" for the forum responses.
forum_df = pd.read_csv("paraphrases.csv")
print(forum_df.head())

# --- Step 3: Generate Embeddings using a Legal Transformer Model ---
# We use a legal-domain transformer model (e.g., LegalBERT) to encode texts.
model_checkpoint = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

def encode_texts(texts, tokenizer, model, max_length=512):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the [CLS] token embedding as a representation
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu()
        embeddings.append(cls_embedding)
    return torch.stack(embeddings)

# Encode legal chunks
print("Encoding legal document chunks...")
legal_embeddings = encode_texts(legal_chunks, tokenizer, model)

# Encode forum responses
forum_texts = forum_df["Humarin_Paraphrase"].tolist()
print("Encoding forum responses...")
forum_embeddings = encode_texts(forum_texts, tokenizer, model)

# --- Step 4: Compute Similarity Scores and Rank Legal Chunks ---
def compute_alignment_score(forum_embedding, legal_embeddings):
    # Compute cosine similarities between a forum response and all legal chunks
    cosine_scores = util.cos_sim(forum_embedding, legal_embeddings)
    # Get the maximum similarity score as a proxy for alignment
    max_score = torch.max(cosine_scores).item()
    return max_score

# Compute alignment scores for all forum responses
alignment_scores = []
for embed in forum_embeddings:
    score = compute_alignment_score(embed.unsqueeze(0), legal_embeddings)
    alignment_scores.append(score)

# forum_df["alignment_scorev"] = alignment_scores[:len(alignment_scores)//2]
forum_df["alignment_scoreh"] = alignment_scores

# Optionally, label a response as "aligned" if the score exceeds a threshold.
# You may adjust the threshold based on your experiments.
threshold = 0.7
# forum_df["predicted_alignmentv"] = forum_df["alignment_scorev"].apply(lambda x: "aligned" if x >= threshold else "not aligned")
forum_df["predicted_alignmenth"] = forum_df["alignment_scoreh"].apply(lambda x: "aligned" if x >= threshold else "not aligned")


print(forum_df.head())

# Save the results
forum_df.to_csv("forum_alignment_results.csv", index=False)

Number of legal chunks: 391
                                  Original_Paragraph  \
0  Preamble\r\nThe licenses for most software are...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if you distribute copies of the l...   

                                  Original_Sentences  \
0  Preamble\r\nThe licenses for most software are...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if you distribute copies of the l...   

                                    Vamsi_Paraphrase  \
0  Preamble The licenses for most software are de...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if

ValueError: Length of values (684) does not match length of index (342)

In [ ]:
forum_df["alignment_scorev"] = alignment_scores[:len(alignment_scores)//2]
forum_df["alignment_scoreh"] = alignment_scores[len(alignment_scores)//2]

# Optionally, label a response as "aligned" if the score exceeds a threshold.
# You may adjust the threshold based on your experiments.
threshold = 0.7
forum_df["predicted_alignmentv"] = forum_df["alignment_scorev"].apply(lambda x: "aligned" if x >= threshold else "not aligned")
forum_df["predicted_alignmenth"] = forum_df["alignment_scoreh"].apply(lambda x: "aligned" if x >= threshold else "not aligned")


print(forum_df.head())

# Save the results
forum_df.to_csv("forum_alignment_results.csv", index=False)

                                  Original_Paragraph  \
0  Preamble\r\nThe licenses for most software are...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if you distribute copies of the l...   

                                  Original_Sentences  \
0  Preamble\r\nThe licenses for most software are...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if you distribute copies of the l...   

                                    Vamsi_Paraphrase  \
0  Preamble The licenses for most software are de...   
1  This license, the Library General Public Licen...   
2  When we speak of free software, we are referri...   
3  To protect your rights, we need to make restri...   
4  For example, if you distribute copies of th